<a href="https://colab.research.google.com/github/Suryanshu-Pugla/INDE-577--Data-Science-Machine-Learning/blob/main/Supervised%20Learning/6.1%20Decision%20Tree%20from%20Scartch/Decision_Tree_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Step 1: Define the Decision Tree Classifier
class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    # Gini Impurity calculation
    def gini_impurity(self, y):
        classes, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        gini = 1 - np.sum(probabilities ** 2)
        return gini

    # Split data into left and right subsets
    def split_data(self, X_column, threshold):
        left_mask = X_column <= threshold
        right_mask = ~left_mask
        return left_mask, right_mask

    # Find the best split for a dataset
    def best_split(self, X, y):
        best_gini = float('inf')
        best_feature = None
        best_threshold = None
        n_features = X.shape[1]

        for feature_idx in range(n_features):
            X_column = X[:, feature_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                left_mask, right_mask = self.split_data(X_column, threshold)
                if np.sum(left_mask) < self.min_samples_split or np.sum(right_mask) < self.min_samples_split:
                    continue

                # Calculate weighted Gini impurity
                gini_left = self.gini_impurity(y[left_mask])
                gini_right = self.gini_impurity(y[right_mask])
                gini_total = (len(y[left_mask]) * gini_left + len(y[right_mask]) * gini_right) / len(y)

                if gini_total < best_gini:
                    best_gini = gini_total
                    best_feature = feature_idx
                    best_threshold = threshold

        return best_feature, best_threshold

    # Recursive tree building
    def build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        # Stopping conditions
        if depth >= self.max_depth or len(unique_classes) == 1 or num_samples < self.min_samples_split:
            leaf_value = self.majority_class(y)
            return {'leaf': True, 'value': leaf_value}

        # Find the best split
        feature_idx, threshold = self.best_split(X, y)
        if feature_idx is None:
            leaf_value = self.majority_class(y)
            return {'leaf': True, 'value': leaf_value}

        # Split the data
        left_mask, right_mask = self.split_data(X[:, feature_idx], threshold)
        left_tree = self.build_tree(X[left_mask], y[left_mask], depth + 1)
        right_tree = self.build_tree(X[right_mask], y[right_mask], depth + 1)

        # Return the node
        return {
            'leaf': False,
            'feature_idx': feature_idx,
            'threshold': threshold,
            'left': left_tree,
            'right': right_tree
        }

    # Find the majority class
    def majority_class(self, y):
        values, counts = np.unique(y, return_counts=True)
        return values[np.argmax(counts)]

    # Train the decision tree
    def fit(self, X, y):
        self.tree = self.build_tree(X, y)

    # Predict for a single sample
    def predict_sample(self, node, sample):
        if node['leaf']:
            return node['value']
        if sample[node['feature_idx']] <= node['threshold']:
            return self.predict_sample(node['left'], sample)
        else:
            return self.predict_sample(node['right'], sample)

    # Predict for all samples
    def predict(self, X):
        return np.array([self.predict_sample(self.tree, sample) for sample in X])


# Step 2: Load Breast Cancer Dataset
data = load_breast_cancer()
X = data['data']  # Feature matrix
y = data['target']  # Target labels (0: Malignant, 1: Benign)

# Step 3: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the Decision Tree
tree = DecisionTree(max_depth=5, min_samples_split=5)
tree.fit(X_train, y_train)

# Step 5: Test and Evaluate the Model
y_pred = tree.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Step 6: Print the Results
print("Decision Tree Classifier Results on Breast Cancer Dataset:")
print(f"Accuracy: {accuracy:.2f}")


Decision Tree Classifier Results on Breast Cancer Dataset:
Accuracy: 0.95
